In [1]:
from IPython.display import HTML

import math
import pandas as pd
import numpy as np

#from scipy.optimize import minimize
#from scipy.optimize import differential_evolution    

from lmfit import Minimizer, Parameters, report_fit

from bokeh.plotting import figure, output_notebook, show
from bokeh.models import LinearAxis, Range1d
from bokeh.models import DatetimeTickFormatter, MonthsTicker, NumeralTickFormatter, Legend


output_notebook()

Loading BokehJS ...

In [2]:
import sys
sys.path.insert(1,'../covid-website')

import SISV as s
from data import Data
from ContactRate import contact_rate


In [3]:
source = 'Johns Hopkins'    
region = 'US'
state = 'New York'

cutoff_positive = 1
cutoff_death = 1
truncate = 0
window = 14

d = Data(source=source, region=region, state=state, county="", cutoff_positive=cutoff_positive, cutoff_death=cutoff_death, truncate=truncate) 


In [5]:
#---------------------------------------------------------------------------------
#transform auxiliary values (which are bounded [0,1] into the breakpoint dates)
def aux_to_breakpoints(aux, x0, xn, minwindow):
    breakpoints = []
    bi1 = x0
    n=len(aux)
    for i,a in enumerate(aux):
        bi = a * (xn-(n-i)*minwindow - (bi1+minwindow)) + bi1+minwindow
        breakpoints.append(bi)
        bi1=bi   
    return breakpoints

#---------------------------------------------------------------------------------
def breakpoints_to_aux(breakpoints, x0, xn, minwindow):
    aux = []
    bi1 = x0
    n=len(breakpoints)
    for i,b in enumerate(breakpoints):
        temp = xn - (n-i)*minwindow - (bi1+minwindow)
        a = 1.0 if temp==0 else (b - bi1 - minwindow) / temp
        aux.append(a)
        bi1=b
    return aux  
    
#aux_to_breakpoints([0.5, 0.5], 0, 100, 10)


#---------------------------------------------------------------------------------


def override(key, overrides, default):
    return default if key not in overrides else overrides[key]

In [6]:
b = aux_to_breakpoints([0.5,0.5,0.5], 5, 100, 10)
a = breakpoints_to_aux(b, 5, 100, 10)
print(b)
print(a)

[42.5, 66.25, 83.125]
[0.5, 0.5, 0.5]


In [ ]:
#---------------------------------------------------------------------------------
def SISV2_lmfit2(d, overrides={}, solver='leastsq'):

    
    
    
    #--------------------------------------
    def merge_params(params, constants, solve_ti=False):
    
        p = params.valuesdict()  #params should be a LMFIT Parameters instance; constants should be a dict
        for i, (k,v) in enumerate(constants.items()):
            if k not in p:  #do not override values that may already be in the Parameters array
                p[k]=v
    
        if solve_ti:  #we are solving for time breakpoints
            #calculate "ti" variables from "auxi" variables
            n = p['segments']
            
            aux = []
            for i in range(1, n+1):
                aux.append(params['aux{}'.format(i-1)])
                
            breakpoints = aux_to_breakpoints(aux, d.x[0], d.x[-1], minwindow)
            
            for i in range(1,n+1):
                #p['beta{}'.format(i)] = params[len(param_list)+i-1]
                p['t{}'.format(i)] = breakpoints[i-1]
                
        return p


    #--------------------------------------
    def lmfit_inner(params, x, constants, column, data=None):
        p = merge_params(params, constants, solve_ti=True if column==s.cF else False)   #solve for time breakpoints Ti through Auxi variables when using fatalities data
        yhat, yyhat = s.SISV_J(x, p)
        if data is None:
            return yhat
        else:
            return yhat[:,column] - data



    #--------------------------------------
    #first stage: calibrate initial infectious population and contact rate over time on fatalities data
    #--------------------------------------
    
        
    gamma = override('gamma', overrides, 1/3)
    segments = override('segments', overrides, 7)
    minwindow = override('minwindow', overrides, 7)

    params = Parameters()
    #(name, value, vary, min, max, expr)
    params.add_many( 
                     ('exp_stages',    1, False),
                     ('inf_stages',    1, False),
                     ('crit_stages',   1, False),
                     ('test_stages',   1, False),
                     
                     ('gamma_exp',      gamma, False),
                     ('gamma',          gamma, False),
                     ('gamma_pos',      1/14, False),
                     ('gamma_crit',     1/14, False),
                     
                     ('death_rate',     0.5e-2, False),
                     ('detection_rate', 5e-2, False),

                     ('population',     d.population, False),
                     ('f0',             1, False),


                     ('immun',          0, False),
                     ('vacc_start',     365, False),
                     ('vacc_rate',      0, False),
                     ('vacc_immun',     1/180, False),
        
                     ('segments',       segments         , False),

                     ('beta0',          7*gamma       , True, 1.1*gamma, 10*gamma),                 
                   )

    for i in range(1, segments+1):  
        params.add('aux{}'.format(i-1),value=0.1, vary=True, min=0, max=1)
        params.add('beta{}'.format(i), value= 1*gamma, vary=True,  min=0.05*gamma, max=10*gamma)

    #lmfit Parameters cannot accept string values so they get passed in a separate argument
    constants = { 
        'interv':'piecewise linear',
        'init_beta':'',
    }

    for idx, (k,v) in enumerate(overrides.items()):
        if k in params:
            params[k].set(value=v)
        else:
            constants[k]=v
            
    fitter = Minimizer(lmfit_inner, params, fcn_args=(d.x, constants, s.cF, d.fatalities))
    result = fitter.minimize(method=solver)

    p = merge_params(result.params, constants, solve_ti=True)  #merge the calibrated variables into the dictionary of params
    #print(p)    
    #-------------------------
    #second stage: calibrate detection rate on positive test results data
    #------------------------

    params = Parameters()
    #(name, value, vary, min, max, expr)
    params.add_many( 
                     ('detection_rate', 15e-2, True, 0, 1),
                   )

    fitter = Minimizer(lmfit_inner, params, fcn_args=(d.x, p, s.cP, d.positives))
    result = fitter.minimize(method=solver)

    p = merge_params(result.params, p, solve_ti=False) #merge the calibrated variables into the dictionary of params




    
    #yy = s.SISV(d.x, p)
    return p
    


In [ ]:
%%time

p1 = SISV_lmfit(d, solver='differential_evolution',
                  overrides={
                      'exp_stages'  : 1,
                      'inf_stages'  : 1,
                      'crit_stages' : 1,
                      'test_stages' : 1,
                      'death_rate'  : 0.73e-2,
                      'gamma_exp'   : 1/4,
                      'gamma'       : 1/4,
                      'gamma_crit'  : 1/17, #1/(17+21),
                      'gamma_pos'   : 1/7,
                      'interv'      :'piecewise linear',
                      'init_beta'   : '',
                      'minwindow'   : 7,
                      'segments'    : 5,
                      'f0'          : 1,
                  })





In [ ]:
y1, yy1 = s.SISV2(d.x, p1) 

def print_params(title, params):
    print('----------')
    print(title)
    print("detect:", params["detection_rate"])
    print("f0:", params["f0"])
    print("R0:", params['beta0']/params['gamma'])
    for i in range(1, 1+params['segments']):
          print(params['t{}'.format(i)],":", params['beta{}'.format(i)]/params['gamma'])

print_params("p1", p1)
#print_params("p2", p2)

print('t0', y1[0, s.cE], y1[0,s.cI], y1[0,s.cC], y1[0, s.cF])

p = figure(title='Initialization', plot_width=800, plot_height=600 , y_axis_type="linear")
p.line(range(180), yy1[:,s.cF], line_width=1, line_color='black', line_dash='solid', alpha=0.7)
show(p)


p = figure(title='Excess Fatalities', plot_width=800, plot_height=600 , y_axis_type="log")
p.y_range.start = 1

r0 = p.line(d.xd[d.minD+1:], d.dfatalities, line_width=1, line_color='red', line_dash='dotted', alpha=0.3)
r1 = p.circle(d.xd[d.minD+1:], d.dfatalities, size=5, color="red", alpha=0.3)

#plot 7-day rolling average
rolling = pd.DataFrame(data = d.dfatalities).interpolate().rolling(7).mean()
r2 = p.line(d.xd[d.minD+1:], rolling.loc[:,0].values, line_width=1, line_color='red')

r3_1 = p.line(d.xd[d.minD+1:], np.diff(y1[d.minD:,s.cF]), line_width=1, line_color='black', line_dash='solid', alpha=0.7)

#-------------

r0_p = p.line(d.xd[d.minP+1:], d.dpositives, line_width=1, line_color='red', line_dash='dotted', alpha=0.3)
r1_p = p.circle(d.xd[d.minP+1:], d.dpositives, size=5, color="red", alpha=0.3)

r3_1_p = p.line(d.xd[d.minP+1:], np.diff(y1[d.minP:,s.cP]), line_width=1, line_color='black', line_dash='solid', alpha=0.7)

legend = [
    ("COVID fatalities"   , [r0, r1]),
    ("COVID 7-day average"   , [r2]),
    
    ("p1", [r3_1]),

    ("COVID positives"   , [r0_p, r1_p]),
    ("p1_p", [r3_1_p]),
]

    
p.add_layout(Legend(items=legend, location='center'), 'right')
   
p.yaxis[0].formatter = NumeralTickFormatter(format="0,0")

p.xaxis.ticker = MonthsTicker(months=list(range(1,13)))

p.xaxis.formatter=DatetimeTickFormatter(
        hours=["%d %B %Y"],
        days=["%d %B %Y"],
        months=["%d %B %Y"],
        years=["%d %B %Y"],
    )

p.xgrid.ticker = p.xaxis.ticker

p.xaxis.major_label_orientation = math.pi/4


p.extra_x_ranges['x2'] = Range1d(d.x[0], d.x[-1])
ax2 = LinearAxis(x_range_name="x2", axis_label="days")
p.add_layout(ax2, 'below')


p.ygrid.grid_line_color = 'navy'
p.ygrid.grid_line_alpha = 0.3
p.ygrid.minor_grid_line_color = 'navy'
p.ygrid.minor_grid_line_alpha = 0.1

p.yaxis[0].axis_label = 'Number of deaths per day'

#p.toolbar_location = None
show(p)

rplot = figure(title='R0')
R0_1 = contact_rate(d.x, p1) / p1['gamma']
rplot.line(d.x, R0_1, line_width=1, line_color='black', line_dash='solid', alpha=1)
show(rplot)
